# Dataset
https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

# Data Preparation
The dataset contains around 1000 images of hairs in the separate folders for training and test sets.
Reproducibility

Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs. Therefore, in this homework we suggest to:

- install tensorflow version 2.17.1
- set the seed generators by:

        import numpy as np
        import tensorflow as tf
        SEED = 42
        np.random.seed(SEED)
        tf.random.set_seed(SEED)

# Model

For this homework we will use **Convolutional Neural Network (CNN)**. Like in the lectures, we'll use Keras.
You need to develop the model with following structure:
- The shape for input should be (200, 200, 3)
- Next, create a convolutional layer (Conv2D):
    - Use 32 filters
    - Kernel size should be (3, 3) (that's the size of the filter)
    - Use 'relu' as activation
- Reduce the size of the feature map with max pooling (MaxPooling2D)
    - Set the pooling size to (2, 2)
- Turn the multi-dimensional result into vectors using a Flatten layer
- Next, add a Dense layer with 64 neurons and 'relu' activation
- Finally, create the Dense layer with 1 neuron - this will be the output
    - The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use SGD with the following parameters:
- SGD(lr=0.002, momentum=0.8)

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

%matplotlib inline

In [46]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [47]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

base_model.trainable = False

x = base_model.output
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

optimizer = SGD(learning_rate=0.002, momentum=0.8)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Question 1

Since we have a binary classification problem, what is the best loss function for us?

- mean squared error
- **binary crossentropy** <-
- categorical crossentropy
- cosine similarity

Note: since we specify an activation for the output layer, we don't need to set from_logits=True

# Question 2

What's the total number of parameters of the model? You can use the summary method for that.
- 896
- 11214912
- 15896912
- **20072512** <-

In [48]:
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 200, 200,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 99, 99,    │        864 │ input_layer_7[0]… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 99, 99,    │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 99, 99,    │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 97, 97,    │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 97, 97,    │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 97, 97,    │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 97, 97,    │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 97, 97,    │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 97, 97,    │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 97, 97,    │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 97, 97,    │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 49, 49,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 49, 49,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 49, 49,    │        512 │ conv2d_33[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_84 (Add)        │ (None, 49, 49,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 49, 49,    │          0 │ add_84[0][0]    

 Total params: 21,459,657 (81.86 MB)

 Trainable params: 598,177 (2.28 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

# Generators and Training

For the next two questions, use the following data generator for both train and test sets:
ImageDataGenerator(rescale=1./255)
- We don't need to do any additional pre-processing for the images.
- When reading the data from train/test directories, check the class_mode parameter. Which value should it be for a binary classification problem?
- Use batch_size=20
- Use shuffle=True for both training and test sets.

For training use .fit() with the following params:

    model.fit(
        train_generator,
        epochs=10,
        validation_data=test_generator
    )

In [49]:
datagen = ImageDataGenerator(rescale=1./255)

In [50]:
train_generator = datagen.flow_from_directory(
    './data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 738 images belonging to 2 classes.


In [51]:
test_generator = datagen.flow_from_directory(
    './data/test',
    target_size=(200,200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 189 images belonging to 2 classes.


# Question 3
What is the median of training accuracy for all the epochs for this model?
- 0.10
- 0.32
- 0.50
- **0.72** <-

In [52]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 34s 838ms/step - accuracy: 0.7346 - loss: 0.4717 - val_accuracy: 0.9259 - val_loss: 0.1913
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 28s 761ms/step - accuracy: 0.9771 - loss: 0.0771 - val_accuracy: 0.9630 - val_loss: 0.1113
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 28s 768ms/step - accuracy: 0.9838 - loss: 0.0549 - val_accuracy: 0.9683 - val_loss: 0.1095
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 29s 776ms/step - accuracy: 0.9999 - loss: 0.0231 - val_accuracy: 0.9577 - val_loss: 0.1102
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 28s 763ms/step - accuracy: 0.9957 - loss: 0.0168 - val_accuracy: 0.9577 - val_loss: 0.1081
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 30s 809ms/step - accuracy: 1.0000 - loss: 0.0120 - val_accuracy: 0.9683 - val_loss: 0.1081
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 29s 780ms/step - accuracy: 1.0000 - loss: 0.0082 - val_accuracy: 0.9630 - val_loss: 0.1085
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 30s 815ms/step - accuracy: 1.0000 - loss: 0.0066 - val_accu

In [53]:
train_accuracy = history.history['accuracy']
median_train_accuracy = np.median(train_accuracy)
print(median_train_accuracy)

0.9993225038051605


# Question 4
What is the standard deviation of training loss for all the epochs for this model?
- **0.028** <-
- 0.068
- 0.128
- 0.168

In [54]:
training_loss = history.history['loss']
median_training_loss = np.median(training_loss)
print(median_training_loss)

0.012589458841830492


# Data Augmentation
For the next two questions, we'll generate more data using data augmentations. Add the following augmentations to your training data generator:
- rotation_range=50,
- width_shift_range=0.1,
- height_shift_range=0.1,
- zoom_range=0.1,
- horizontal_flip=True,
- fill_mode='nearest'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Test data generator (without augmentation, only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

In [58]:
train_generator = train_datagen.flow_from_directory(
    './data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',  # For binary classification
    shuffle=True
)

Found 738 images belonging to 2 classes.


In [59]:
test_generator = test_datagen.flow_from_directory(
    './data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',  # For binary classification
    shuffle=True
)

Found 189 images belonging to 2 classes.


# Question 5
Let's train our model for 10 more epochs using the same code as previously.
    Note: make sure you don't re-create the model - we want to continue training the model we already started training.
What is the mean of test loss for all the epochs for the model trained with augmentations?
- **0.26** <-
- 0.56
- 0.86
- 1.16

In [62]:
history = model.fit(
    train_generator,
    epochs=20,  # continue training for more epochs (total 20)
    initial_epoch=10,  # start from the 11th epoch
    validation_data=test_generator
)

Epoch 11/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 610ms/step - accuracy: 0.9278 - loss: 0.1915

c:\Users\pgdem\.virtualenvs\homework_08-FZqzIBsR\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 29s 781ms/step - accuracy: 0.9283 - loss: 0.1905 - val_accuracy: 0.9418 - val_loss: 0.1943
Epoch 12/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 30s 798ms/step - accuracy: 0.9550 - loss: 0.1079 - val_accuracy: 0.9259 - val_loss: 0.2314
Epoch 13/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 30s 820ms/step - accuracy: 0.9683 - loss: 0.0702 - val_accuracy: 0.9577 - val_loss: 0.1142
Epoch 14/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 29s 774ms/step - accuracy: 0.9696 - loss: 0.0687 - val_accuracy: 0.9630 - val_loss: 0.1138
Epoch 15/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 32s 859ms/step - accuracy: 0.9898 - loss: 0.0297 - val_accuracy: 0.9577 - val_loss: 0.1181
Epoch 16/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 30s 812ms/step - accuracy: 0.9715 - loss: 0.0691 - val_accuracy: 0.9577 - val_loss: 0.1317
Epoch 17/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 29s 780ms/step - accuracy: 0.9869 - loss: 0.0427 - val_accuracy: 0.9630 - val_loss: 0.1196
Epoch 18/20
37/37 ━━━━━━━━━━━━━━━━━━━━ 31s 849ms/step - accuracy: 0.9763 - loss: 0.0490 - val_accuracy

In [63]:
test_loss = history.history['val_loss']
mean_test_loss = np.mean(test_loss)
print(mean_test_loss)

0.14016400128602982


# Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?
- 0.31
- 0.51
- 0.71
- **0.91** <-

In [65]:
test_accuracy = history.history['val_accuracy']
test_accuracy_last_5 = test_accuracy[5:10]
average_test_accuracy = np.mean(test_accuracy_last_5)
print(average_test_accuracy)

0.958730149269104
